<a href="https://colab.research.google.com/github/khushishah03/Mini-project-Smartknower/blob/master/covid_19_time_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np 
import pandas as pd
import pandas.util.testing as tm
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import datetime as dt
from datetime import timedelta

from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error,mean_squared_log_error, r2_score, make_scorer
from sklearn.preprocessing import PolynomialFeatures

import scipy.cluster.hierarchy as sch

import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf


import plotly.express as px
import plotly.offline as py
import plotly.graph_objs as go
from plotly.subplots import make_subplots
py.init_notebook_mode(connected= True)


import warnings
warnings.filterwarnings('ignore')

In [3]:
data = pd.read_csv("/content/drive/My Drive/Projects/dataset/COVID-19 State wise Cases.csv")

In [5]:
data.head()

,S. No.,Date,Region,Confirmed Cases,Active Cases,Cured/Discharged,Death
0,1,03-27-2020,Andaman and Nicobar Islands,1,1,0.0,0.0
1,2,03-28-2020,Andaman and Nicobar Islands,6,6,0.0,0.0
2,3,03-29-2020,Andaman and Nicobar Islands,9,9,0.0,0.0
3,4,03-30-2020,Andaman and Nicobar Islands,9,9,0.0,0.0
4,5,03-31-2020,Andaman and Nicobar Islands,10,10,0.0,0.0


In [6]:
data['Date']= pd.to_datetime(data['Date'])
data.head(5)

,S. No.,Date,Region,Confirmed Cases,Active Cases,Cured/Discharged,Death
0,1,2020-03-27,Andaman and Nicobar Islands,1,1,0.0,0.0
1,2,2020-03-28,Andaman and Nicobar Islands,6,6,0.0,0.0
2,3,2020-03-29,Andaman and Nicobar Islands,9,9,0.0,0.0
3,4,2020-03-30,Andaman and Nicobar Islands,9,9,0.0,0.0
4,5,2020-03-31,Andaman and Nicobar Islands,10,10,0.0,0.0


In [7]:
print("Dataset Description")
print("Earliest Entry: ",data['Date'].min())
print("Last Entry:    ",data['Date'].max())
print("Total Days:    ",(data['Date'].max() - data['Date'].min()))

Dataset Description
Earliest Entry:  2020-03-12 00:00:00
Last Entry:     2020-10-10 00:00:00
Total Days:     212 days 00:00:00


In [8]:
#statewise cases 12-03-2020 to 10-10-2020
statewise= data.groupby(['Region','Date']).agg({"Confirmed Cases":'sum',"Cured/Discharged":'sum',"Death":'sum', 'Active Cases':'sum'})
statewise

Confirmed Cases  ...  Active Cases
Region                      Date                         ...              
Andaman and Nicobar Islands 2020-03-27                1  ...             1
                            2020-03-28                6  ...             6
                            2020-03-29                9  ...             9
                            2020-03-30                9  ...             9
                            2020-03-31               10  ...            10
...                                                 ...  ...           ...
West Bengal                 2020-10-06           273679  ...         27717
                            2020-10-07           277049  ...         27988
                            2020-10-08           280504  ...         28361
                            2020-10-09           284030  ...         28854
                            2020-10-10           287603  ...         29296

[7165 rows x 4 columns]

In [9]:
activecases=statewise['Active Cases'].sum()
cured=statewise['Cured/Discharged'].sum()
death=statewise['Death'].sum()

labels=['Active Cases','Cured/Discharged','Death']
values=[activecases,cured,death]
fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
fig.update_layout(title=" Pie chart of Different types of cases")
fig.show() 

In [10]:
#datawise cases from 12-03-2020 to 10-10-2020
datewise= data.groupby(['Date']).agg({"Confirmed Cases":'sum',"Cured/Discharged":'sum',"Death":'sum',"Active Cases":'sum'})
datewise.tail()

,Confirmed Cases,Cured/Discharged,Death,Active Cases
Date,,,,
2020-10-06,6685082,5662490.0,103569.0,919023
2020-10-07,6757131,5744693.0,104555.0,907883
2020-10-08,6835655,5827704.0,105526.0,902425
2020-10-09,6906151,5906069.0,106490.0,893592
2020-10-10,6979423,5988822.0,107416.0,883185


In [11]:
print("No. of States/UT suffering: ",len(data['Region'].unique()))
print("Total Confirmed Cases in India:  ",datewise['Confirmed Cases'].iloc[-1])
print("Total Recovered Cases in India:  ",datewise['Cured/Discharged'].iloc[-1])
print("Total Death Cases in India:  ",datewise['Death'].iloc[-1])

No. of States/UT suffering:  37
Total Confirmed Cases in India:   6979423
Total Recovered Cases in India:   5988822.0
Total Death Cases in India:   107416.0


In [12]:
fig= go.Figure()

fig.add_trace(go.Scatter(x=datewise.index, y=datewise['Confirmed Cases'], mode='lines+markers', name='Confirmed Cases'))
fig.add_trace(go.Scatter(x=datewise.index, y=datewise['Cured/Discharged'], mode='lines+markers', name='Cured Cases'))
fig.add_trace(go.Scatter(x=datewise.index, y=datewise['Death'], mode='lines+markers', name='Death Cases'))

fig.update_layout(title=" Growth of Different types of cases", xaxis_title="Date", yaxis_title="Number of Cases", legend= dict(x=0, y=1, traceorder="normal"))
fig.show()

In [13]:
specs = [[{'type':'domain'}, {'type':'domain'}],[{'type':'domain'}, {'type':'domain'}]]
fig = make_subplots(rows=2, cols=2, specs=specs, subplot_titles=["Maharashtra","Andra Pradesh","Karnataka",
"Tamil Nadu"])

#Maharashtra
Maharashtra_active = statewise['Active Cases'].iloc[0]
Maharashtra_cured = statewise['Cured/Discharged'].iloc[0]
Maharashtra_death = statewise['Death'].iloc[0]

labels=['Active Cases','Cured/Discharged','Death']
values_maharashtra=[Maharashtra_active,Maharashtra_cured,Maharashtra_death]
fig.add_trace(go.Pie(labels=labels, values=values_maharashtra), 1, 1)

#Andrapradesh
ap_active = statewise['Active Cases'].iloc[1]
ap_cured =statewise['Cured/Discharged'].iloc[1]
ap_death =statewise['Death'].iloc[1]

labels=['Active Cases','Cured/Discharged','Death']
values_ap=[ap_active,ap_cured,ap_death]
fig.add_trace(go.Pie(labels=labels, values=values_ap), 1, 2)

#Karnataka
karnataka_active = statewise['Active Cases'].iloc[2]
karnataka_cured = statewise['Cured/Discharged'].iloc[2]
karnataka_death = statewise['Death'].iloc[2]

labels=['Active Cases','Cured/Discharged','Death']
values_karnataka=[karnataka_active,karnataka_cured,karnataka_death]
fig.add_trace(go.Pie(labels=labels, values=values_karnataka), 2, 1)

#Tamilnadu
tn_active = statewise['Active Cases'].iloc[3]
tn_cured = statewise['Cured/Discharged'].iloc[3]
tn_death = statewise['Death'].iloc[3]

labels=['Active Cases','Cured/Discharged','Death']
values_tn=[tn_active,tn_cured,tn_death]
fig.add_trace(go.Pie(labels=labels, values=values_tn), 2, 2)

fig.update(layout_title_text='State wise pie chart of different types of cases',layout_showlegend=False)
fig = go.Figure(fig)
fig.show()

In [14]:
specs = [[{'type':'domain'}, {'type':'domain'}],[{'type':'domain'}, {'type':'domain'}]]
fig = make_subplots(rows=2, cols=2, specs=specs, subplot_titles=["Uttar Pradesh","Delhi","West bengal","Kerela"])

#Uttarpradesh
up_active = statecases['Active Cases'].iloc[4]
up_cured = statecases['Cured/Discharged'].iloc[4]
up_death = statecases['Death'].iloc[4]

labels=['Active Cases','Cured/Discharged','Death']
values_up=[up_active,up_cured,up_death]
fig.add_trace(go.Pie(labels=labels, values=values_up), 1, 1)

#Delhi
delhi_active = statecases['Active Cases'].iloc[5]
delhi_cured = statecases['Cured/Discharged'].iloc[5]
delhi_death = statecases['Death'].iloc[5]

labels=['Active Cases','Cured/Discharged','Death']
values_delhi=[delhi_active,delhi_cured,delhi_death]
fig.add_trace(go.Pie(labels=labels, values=values_delhi), 1, 2)

#west bengal
wb_active = statecases['Active Cases'].iloc[6]
wb_cured = statecases['Cured/Discharged'].iloc[6]
wb_death = statecases['Death'].iloc[6]

labels=['Active Cases','Cured/Discharged','Death']
values_wb=[wb_active,wb_cured,wb_death]
fig.add_trace(go.Pie(labels=labels, values=values_wb), 2, 1)

#kerela
kerela_active = statecases['Active Cases'].iloc[7]
kerela_cured = statecases['Cured/Discharged'].iloc[7]
kerela_death = statecases['Death'].iloc[7]

labels=['Active Cases','Cured/Discharged','Death']
values_kerela=[kerela_active,kerela_cured,kerela_death]
fig.add_trace(go.Pie(labels=labels, values=values_kerela), 2, 2)


fig.update(layout_title_text='State wise pie chart of different types of cases',layout_showlegend=False)
fig = go.Figure(fig)
fig.show()

NameError: ignored